In [19]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os
from osgeo import gdal
import glob

In [2]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET

In [3]:
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the bucket.")

2023-06-01.csv
2023-06-02.csv
STAC_LiDAR/
STAC_LiDAR/DEM/
STAC_LiDAR/DEM/bc_092o018_xli1m_utm10_2018 (2).tif
STAC_LiDAR/DEM/bc_092o018_xli1m_utm10_2018.tif
STAC_LiDAR/DEM/bc_092o019_xli1m_utm10_2018 (2).tif
STAC_LiDAR/DEM/bc_092o019_xli1m_utm10_2018.tif
STAC_LiDAR/DEM/bc_092o020_xli1m_utm10_2018 (2).tif
STAC_LiDAR/DEM/bc_092o020_xli1m_utm10_2018.tif
STAC_LiDAR/DEM/bc_092o028_xli1m_utm10_2018.tif
STAC_LiDAR/DEM/bc_092o029_xli1m_utm10_2019.tif
STAC_LiDAR/DEM/bc_092o030_xli1m_utm10_2018.tif
STAC_LiDAR/JSON/bc_092o018_3_2_4_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_3_4_2_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_3_4_4_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_4_1_3_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_4_1_4_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_4_3_1_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_4_3_2_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_4_3_3_xyes_12_utm10_2018.json
STAC_LiDAR/JSON/bc_092o018_4_3_4_xyes_12_utm10_2018

In [4]:
#Create Folder Like Structure(empty objects) for STAC Lidar base area and point clouds, if they do no exist already  .laz files will go into point clouds object
base_obj="STAC_LiDAR/"
pc_obj="STAC_LiDAR/DEM/"
obj_list=[base_obj,pc_obj]

for item in obj_list:
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=item)
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'] == item:
                print(f"The object {item}' in bucket '{bucket}' exists.")
                break
        else:
            print(f"The object {item}' in bucket '{bucket}' dose not exist, adding object")
            s3_client.put_object(Bucket=bucket, Key=item)


# s3_client.put_object(Bucket=bucket, Key=pc_obj)

The object STAC_LiDAR/' in bucket 'rczimv' exists.
The object STAC_LiDAR/DEM/' in bucket 'rczimv' exists.


In [9]:
# specify the folder path (in the current working directory) where objects are/will be
# if the folder doesn't exist, create it
save_path = (r'/home/cfolkers/STAC_LiDAR/DEM')
if not os.path.exists(save_path):
    os.makedirs(save_path)


In [18]:
DEM_list=os.listdir(save_path)

options = ['COMPRESS=LZW', 'NUM_THREADS=ALL_CPUS', 'BIGTIFF=YES']


for f in DEM_list:
    f=os.path.join(save_path,f)
    splits=f.split('.')
    output_cog=f"{splits[0]}_cog.{splits[1]}"
    input_dataset = gdal.Open(f)
    gdal.Translate(output_cog, input_dataset, format='COG', creationOptions=options)
    input_dataset = None  


Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS
Warning 6: driver COG does not support creation option COPY_SRC_OVERVIEWS


['bc_092o020_xli1m_utm10_2018.tif', 'bc_092o020_xli1m_utm10_2018 (2).tif', 'bc_092o018_xli1m_utm10_2018 (2)_cog.tif', 'bc_092o018_xli1m_utm10_2018_cog.tif', 'bc_092o030_xli1m_utm10_2018.tif', 'bc_092o029_xli1m_utm10_2019_cog.tif', 'bc_092o029_xli1m_utm10_2019.tif', 'bc_092o019_xli1m_utm10_2018.tif', 'bc_092o028_xli1m_utm10_2018.tif', 'bc_092o020_xli1m_utm10_2018_cog.tif', 'bc_092o018_xli1m_utm10_2018 (2).tif', 'bc_092o018_xli1m_utm10_2018.tif', 'bc_092o030_xli1m_utm10_2018_cog.tif', 'bc_092o019_xli1m_utm10_2018 (2).tif', 'bc_092o019_xli1m_utm10_2018 (2)_cog.tif', 'bc_092o028_xli1m_utm10_2018_cog.tif', 'bc_092o019_xli1m_utm10_2018_cog.tif', 'bc_092o020_xli1m_utm10_2018 (2)_cog.tif']


In [31]:
# put .laz files into your object storage "STAC_LiDAR/PointClouds/"

DEM_list= glob.glob(fr"{save_path}/*_cog.*")
for obj in DEM_list:
    obj_load=os.path.join(save_path,obj)
    obj_key=pc_obj+obj
    s3_client.upload_file(obj_load, Bucket=bucket, Key=obj_key)
    
print(f"Objects uploaded to folder '{pc_obj}' in bucket '{bucket}'.")


Objects uploaded to folder 'STAC_LiDAR/DEM/' in bucket 'rczimv'.


In [32]:
#list .laz objects in bucket
# object_key="STAC_LiDAR/PointClouds/"
laz_objects=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=pc_obj, StartAfter=pc_obj)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        laz_objects.append(obj['Key'])
        print(obj['Key'])
        print(f"Object Size {obj['Size']}")
else:
    print("No objects found in the bucket.")

STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o018_xli1m_utm10_2018 (2)_cog.tif
Object Size 27945230
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o018_xli1m_utm10_2018_cog.tif
Object Size 131166704
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o019_xli1m_utm10_2018 (2)_cog.tif
Object Size 281923918
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o019_xli1m_utm10_2018_cog.tif
Object Size 256676853
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o020_xli1m_utm10_2018 (2)_cog.tif
Object Size 19758214
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o020_xli1m_utm10_2018_cog.tif
Object Size 370620362
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o028_xli1m_utm10_2018_cog.tif
Object Size 53380148
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o029_xli1m_utm10_2019_cog.tif
Object Size 20150088
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o030_xli1m_utm10_2018_cog.tif
Object Size 80279565


In [8]:
### Run only to delete all objects 

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=pc_obj, StartAfter=pc_obj)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        laz_objects.append(obj['Key'])
        print(obj['Key'])
        s3_client.delete_object(Bucket=bucket, Key=obj['Key']
    )